# Shark League Score
## Import Python libraries

In [ ]:
import pandas as pd
import numpy as np
import logging
import swc_simple_client as swc

## Configure logging

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='shark_notebook.log', level=logging.INFO,)

## Setup notebook variables

In [ ]:
base_url = "https://super-duper-palm-tree-g4v9vrvxx9vgc9jqv-8000.app.github.dev"

##GEt Max Scores